# Building a Crew AI Agentic Application

In [2]:
from crewai import Agent, Crew, Process, Task
import re

In [ ]:
%store -r

In [3]:
max_iterations = 2

In [4]:
%load_ext dotenv
%dotenv ../.env

In [ ]:
import boto3
from langchain_aws import ChatBedrockConverse
from typing import Optional

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

In [ ]:
import sys
import uuid

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
kb = KnowledgeBasesForAmazonBedrock()

knowledge_base_name = f'pr-agent-kb-{str(uuid.uuid4())[:5]}'
knowledge_base_description = "KB containing information about pristine PR articles"
s3_bucket_name = f"labs-bucket-{region}-{account_id}"
bucket_prefix = "data/kb/reflection/"

kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    s3_bucket_name,
    "amazon.titan-embed-text-v2:0",
    bucket_prefix
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

In [ ]:
def upload_directory(path, bucket_name, bucket_prefix):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,f"{bucket_prefix}{file}")

In [ ]:
upload_directory("../good_prs", s3_bucket_name, bucket_prefix)

In [5]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools.aws.bedrock.knowledge_base.retriever_tool import BedrockKBRetrieverTool

# Initialize the tool
kb_tool = BedrockKBRetrieverTool(
    knowledge_base_id=kb_id,
    number_of_results=3
)

kb_tool.description = """Retrieves examples of pristine, high quality PR articles that have been published in the past."""

In [6]:
from crewai.tools import tool

@tool("PR article review guidelines")
def pr_review_guidelines() -> str:
    """Returns the editorial guidelines for a PR article."""
    with open("pr_review_guidelines.txt", "r") as f:
        guidelines = f.read()
        return guidelines

# PR Writing Agent

In [7]:
@tool("PR article writing best practices")
def pr_writing_best_practices() -> str:
    """Returns the editorial best practices for writing PR articles."""
    with open("pr_article_writing_best_practices.txt", "r") as f:
        best_practices = f.read()
        return best_practices

In [8]:
pr_writing_task_description = """You are given an outline to write a PR atricle. 
You have access to a best practice tool to find the best practices for writing the article.
You also have access to a knowledge base tool that contains some examples of pristine, high quality PRs that have been published in the past. 
You should use the knowledge base to help with the style and structure for the article.

Here's the article outline:

{article_outline}

"""

# Configure Bedrock LLM
pr_writer_llm = LLM(
    model="bedrock/us.amazon.nova-premier-v1:0")

# Create agents
pr_writer_agent = Agent(
    role='PRWriter',
    goal='Write a PR article from an outline, or rewriting an article based on some feedback from a reviewer.',
    backstory='You are a professional media content writer. You are good at writing PR articles for an upcoming show/movie based on an outline of the article',
    llm = pr_writer_llm,
    tools=[kb_tool, pr_writing_best_practices],
    verbose=True,
    max_iter=max_iterations
)

pr_writing_task = Task(
    description=pr_writing_task_description,
    expected_output="""A fully written article in markdown format""",
    # agent=pr_writer_agent,
    async_execution=False)

# PR Reviewer Agent

In [9]:
# Configure Bedrock LLM
pr_reviewer_llm = LLM(
    model="bedrock/us.amazon.nova-pro-v1:0")

# Create agents
pr_review_agent = Agent(
    role='PRReviewer',
    goal='Provide feedback to improve the given PR article reviewer',
    backstory='You are a professional media content analyst. You are an expert in reviewing PR articles to ensure the specific guidelines and standards are met.',
    llm = pr_reviewer_llm,
    verbose=True,
    tools=[pr_review_guidelines],
    max_iter=max_iterations
)

pr_reviewer_task_description = """Performs thorough article review to ensure specific guidelines and quality are met.

You have access to a reviewer guideline tool that contains the specific guidelines to follow.
"""

pr_review_task = Task(
    description=pr_reviewer_task_description,
    expected_output="""Detailed feedback on how to improve the given article. If the article is good enough, reply with 'quality met. Good to go!' """,
    # agent=pr_review_agent,
    async_execution=False,
    context=[pr_writing_task])



# PR Rewriting Agent

In [10]:
# Configure Bedrock LLM
pr_rewriter_llm = LLM(
    model="bedrock/us.amazon.nova-premier-v1:0")
# Create agents
pr_rewriter_agent = Agent(
    role='PRRewriter',
    goal='Rewrite an article based on the given feedback.',
    backstory="""You are a professional media content writer. You are good at writing PR articles for an upcoming show/movie based on an outline of the article.
You are to rewrite the PR article by incorporating the given review feedback.""",
    llm = pr_rewriter_llm,
    tools = [pr_writing_best_practices],
    verbose=True,
    max_iter=max_iterations
)

pr_rewriting_task_description = """Rewrite the PR article by incorporating the given review feedback."""
pr_rewriting_task = Task(
    description=pr_rewriting_task_description,
    expected_output="""A fully written article in markdown format""",
    # agent=pr_rewriter_agent,
    async_execution=False,
    context=[pr_writing_task, pr_review_task])

# Supervisor Agent  

In [ ]:
supervisor_agent_backstory = """You are a PR article orchestration agent. Your ONLY role is to manage the workflow by activating the appropriate agent at the appropriate time. You are NOT responsible for evaluating the quality or content of any agent's work.

Here are the list of agents available to you: 

1. PRWriter - Writes a PR article based on the given article outline.

2. PRReviewer - Reviews the PR article generated by the PR Writer to provide recommendation to improve the PR article.

3. PRRewriter - Rewrite a PR article based on the given feedback

You job is to delegate the writng, reviewing and rewriting tasks to the given tools. 

Here are the IMPORTANT guidelines:
- You must first write a draft article using the writing tool before performing any reviews. 
- Keep a count of the number of iterations of writing and review. You must only iterate the PR writing and review for maximum 2 times. You must not do this for more than 2 times. 
- The final draft of the PR must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final PR article in markdown format.

"""

# Configure Bedrock LLM
supervisor_agent_llm = LLM(
    model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0",
    # model="bedrock/us.amazon.nova-premier-v1:0",
    temperature=0.1,
    top_p=0.99)

# Define the manager agent
supervisor_agent = Agent(
    role="PR article orchestration manager",
    goal="Efficiently orchestrates the crew to ensure high-quality PR article creation",
    backstory=supervisor_agent_backstory,
    allow_delegation=True,
    llm=supervisor_agent_llm,
    max_iter=max_iterations
)


In [12]:

crew = Crew(
    agents=[pr_review_agent, pr_rewriter_agent, pr_writer_agent],
    tasks=[pr_writing_task, pr_review_task, pr_rewriting_task],
    manager_agent=supervisor_agent,
    process=Process.hierarchical,
    # manager_llm=supervisor_agent_llm,
    # chat_llm=supervisor_agent_llm,
    # planning=True,
    # planning_llm=supervisor_agent_llm,
    # memory=True,
    verbose=True)

In [13]:
query = """Write a PR article for an upcoming movie. Here's an outline: 

Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson 
"""

inputs = {
    'article_outline': query
    }

In [ ]:
pr_article_generation_result = crew.kickoff(inputs=inputs)

In [ ]:
print(pr_article_generation_result)